<a href="https://colab.research.google.com/github/ozgurugurr/fasttext_tf_-df/blob/master/tf_%C4%B1df_fasttext.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Google Drive'ı bağlayın ve kodları çalıştırın

In [0]:
%cd drive/My\ Drive

In [0]:
%mkdir tf-ıdf-fasttext
%cd tf-ıdf-fasttext

In [21]:
import os
import spacy
import pickle
from spacy.lang.en import English
from spacy import displacy
import numpy as np
import tqdm
import pandas as pd
from gensim.models import FastText
from gensim.models.phrases import Phrases, Phraser
from gensim.parsing.preprocessing import remove_stopwords, strip_punctuation, strip_non_alphanum
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import xx_ent_wiki_sm

## ENGLISH
# !python -m spacy download en_core_web_md

## MULTILANGUAGE
!python -m spacy download xx_ent_wiki_sm
nlp = xx_ent_wiki_sm.load()

✔ Download and installation successful
You can now load the model via spacy.load('xx_ent_wiki_sm')


In [0]:
from google.colab import files
uploaded = files.upload()

In [0]:
pd.options.display.max_colwidth = 800
path = "/content/drive/My Drive/tf-ıdf-fasttext/AllMRLineItems_Mapping_Matched.csv"
combined = pd.read_csv(path, skip_blank_lines= False, sep=',')
combined = pd.DataFrame(data = combined)

In [0]:
!head -n 100 "/content/drive/My Drive/tf-ıdf-fasttext/AllMRLineItems_Mapping_Matched.csv" > "/content/drive/My Drive/tf-ıdf-fasttext/AllMRLineItems_Mapping_Matched_100.csv"

In [0]:
combined = combined[['MRCreationDate', 'MRID', 'MRTitle', 'MRRemarks', 'MRPrivateNotes',
                    'CommodityCode', 'MRType', 'JobNumber', 'Company', 'MRLineItemID',
                    'MRLineItemDescription', 'Quantity', 'MaterialType', 'Unit',
                    'CostCodeDescription', 'CommonCostCode', 'CommonCostCodeDescription',
                    'ProjectID', 'ProjectCode', 'Flag', 'Prefix',
                    'CategoryCode', 'CategoryCodeName']]

In [0]:
cols = combined.columns.tolist()
cols = ['Flag',
        'ProjectCode',
        'ProjectID',       
        'MRID',
        'MRLineItemID',
        'MRTitle',
        'MRLineItemDescription',
        'CategoryCode',
        'CategoryCodeName',
        'CostCodeDescription',
        'CommonCostCode',
        'CommonCostCodeDescription',
        'MRRemarks',
        'MRPrivateNotes',
        'CommodityCode',
        'MRType',
        'JobNumber',
        'Company',     
        'Quantity',
        'MaterialType',
        'Unit',              
        'Prefix'
        ]
combined = combined[cols]

In [138]:
combined.head()

,Flag,ProjectCode,ProjectID,MRID,MRLineItemID,MRTitle,MRLineItemDescription,CategoryCode,CategoryCodeName,CostCodeDescription,CommonCostCode,CommonCostCodeDescription,MRRemarks,MRPrivateNotes,CommodityCode,MRType,JobNumber,Company,Quantity,MaterialType,Unit,Prefix
0,1,2026,90,31163,348608,Roofing Insulation-DOW ROOFMATE(TM) SL-X,DOW ROOFMATE(TM) SL-X Extruded Polystyrene Foam 50X600X1250MM,ATRX,"Roofing, Self-Adhered Modified Bituminous Membrane",Hot Rubberized Protected Membrane Roofing Assembly,33ATRX,ATRX - Roofing Self-Adhered Modified Bituminous Membrane,Null,Null,A000,MR,02026-02026,-,17500000.0,Null,m3-Cubic meter,33
1,1,2026,90,77639,1023998,75216 SBS Modified Bituminous Membrane Roofing - Cap/Base Sheet for UTL,11008 AQUADERE,ATRX,"Roofing, Self-Adhered Modified Bituminous Membrane",Hot Rubberized Protected Membrane Roofing Assembly,33ATRX,ATRX - Roofing Self-Adhered Modified Bituminous Membrane,Null,Null,A000,MR,02026-02026,TR001-ENKA İnşaat ve Sanayi A.Ş.,2500000.0,Permanent Material,l-Liter,33
2,1,2026,90,77639,1023999,75216 SBS Modified Bituminous Membrane Roofing - Cap/Base Sheet for UTL,54766 SOPRALENE S FLAM 180-40 (base / cap sheet),ATRX,"Roofing, Self-Adhered Modified Bituminous Membrane",Hot Rubberized Protected Membrane Roofing Assembly,33ATRX,ATRX - Roofing Self-Adhered Modified Bituminous Membrane,Null,Null,A000,MR,02026-02026,TR001-ENKA İnşaat ve Sanayi A.Ş.,15040000.0,Permanent Material,sqm-Square meter,33
3,1,2026,90,86556,1148101,75216 - Extra Surface Granule Materials - Soprema,Soprema Paillettes D'ardoise - Gris Chagall,ATRX,"Roofing, Self-Adhered Modified Bituminous Membrane",Hot Rubberized Protected Membrane Roofing Assembly,33ATRX,ATRX - Roofing Self-Adhered Modified Bituminous Membrane,Null,Null,A000,MR,02026-02026,TR001-ENKA İnşaat ve Sanayi A.Ş.,1200000.0,Permanent Material,kg-Kilogram,33
4,1,2026,90,86556,1148102,75216 - Extra Surface Granule Materials - Soprema,Soprema Paillettes D'ardoise - Ardoise Gris,ATRX,"Roofing, Self-Adhered Modified Bituminous Membrane",Hot Rubberized Protected Membrane Roofing Assembly,33ATRX,ATRX - Roofing Self-Adhered Modified Bituminous Membrane,Null,Null,A000,MR,02026-02026,TR001-ENKA İnşaat ve Sanayi A.Ş.,1200000.0,Permanent Material,kg-Kilogram,33


In [0]:
# içinde bir kelime listesi bulunan dokümanların bir listesini oluşturur:
# örnek : ['wc', 'exhaust', 'fan', 'v', '180m3', 'h', 'h', '100', 'pa']
text = []
for i in combined.MRLineItemDescription.values:
  doc = nlp(remove_stopwords(strip_punctuation(strip_non_alphanum(str(i).lower()))))
  tokens = [token.text for token in doc]
  text.append(tokens)

In [0]:
common_terms = ["of", "with", "without", "and", "or", "the", "a"]
# Cümle listesinden ilgili ifadeleri oluşturun:
phrases = Phrases(text, common_terms=common_terms, threshold = 10, min_count=5)
# Phraser nesnesi cümleleri dönüştürmek için kullanılıyor
bigram = Phraser(phrases)
tokens = list(bigram[text])

In [0]:
# FastText modelini eğitin
model = FastText(tokens, size=100, window=3, min_count=1, iter=10, sorted_vocab=1)

In [0]:
!head -n 80000 "/content/drive/My Drive/tf-ıdf-fasttext/AllMRLineItems_Mapping_Matched.csv" > "/content/drive/My Drive/tf-ıdf-fasttext/AllMRLineItems_Mapping_Matched_Test.csv"

In [0]:
# def print_results(N, p, r):
#     print("N\t" + str(N))
#     print("P@{}\t{:.3f}".format(1, p))
#     print("R@{}\t{:.3f}".format(1, r))

# print_results(*model.test('/content/drive/My Drive/tf-ıdf-fasttext/AllMRLineItems_Mapping_Matched_Test.csv'))

In [157]:
model.wv.most_similar('fasteners')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('tpfasteners', 0.9931657314300537),
 ('fastenersthese', 0.980890691280365),
 ('fastenersst', 0.9805139303207397),
 ('fasteners_ec', 0.9770998954772949),
 ('fastenersthis', 0.9761537313461304),
 ('tabs_fasteners', 0.9714223146438599),
 ('fasteners_sx5', 0.9680678844451904),
 ('fasteners_sxc', 0.9680439233779907),
 ('fasteners_sx14', 0.9606038331985474),
 ('fastners', 0.9529014825820923)]

In [149]:
model.wv.similarity('fastener', 'fast')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.7136766

In [146]:
model.wv.most_similar('polystyrene')      

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('polystyrene_foam', 0.9640012383460999),
 ('polythene', 0.9502295255661011),
 ('polypropylene', 0.9262514710426331),
 ('polythelene', 0.9242044687271118),
 ('polysil', 0.9225318431854248),
 ('polypol', 0.9212021827697754),
 ('polyproplene', 0.9208230972290039),
 ('polysyrlyn', 0.9153357148170471),
 ('polypropilen', 0.914897620677948),
 ('polyfoam', 0.9125882387161255)]

In [150]:
model.wv.similarity('polystyrene', 'polystyren')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.9777656

In [0]:
# # kelime vektör modeli kaydet
# model.save_model("/content/drive/My Drive/tf-ıdf-fasttext/model.bin")

In [0]:
# # kelime vektör modeli yükle
# model = fasttext.load_model("/content/drive/My Drive/tf-ıdf-fasttext/model.bin")

In [35]:
# TF-IDF fasttext 'model' ile birlikte kelimeler ve dokümanlar için bir liste listesine ihtiyaç duyar.
from tqdm import tqdm

text = []
for i in tqdm(tokens):
  string = ' '.join(i)
  text.append(string)
tf_idf_vect = TfidfVectorizer(stop_words=None)
final_tf_idf = tf_idf_vect.fit_transform(text)
tfidf_feat = tf_idf_vect.get_feature_names()

100%|██████████| 434598/434598 [00:00<00:00, 1877068.39it/s]


In [36]:
# np.seterr(divide='ignore', invalid='ignore') ??
tfidf_sent_vectors = []; 
row=0;
errors=0
for sent in tqdm(tokens): 
    sent_vec = np.zeros(100) 
    weight_sum =0; 
    for word in sent: 
        try:
            vec = model.wv[word]
            # cümle / inceleme içindeki bir kelimenin tf_idfidf değerini elde etmek
            tfidf = final_tf_idf [row, tfidf_feat.index(word)]
            sent_vec += (vec * tfidf)
            weight_sum += tfidf
        except:
            errors =+1
            pass
    sent_vec /= weight_sum
    #print(np.isnan(np.sum(sent_vec))) ??

    tfidf_sent_vectors.append(sent_vec)
    row += 1
print('errors noted: '+str(errors))

100%|██████████| 434598/434598 [1:27:12<00:00, 83.05it/s]

errors noted: 1


In [0]:
with open("/content/drive/My Drive/tf-ıdf-fasttext/tfidf_sent_vectors.pkl", 'wb') as handle:
                    pickle.dump(tfidf_sent_vectors, handle)

In [0]:
# bu vektörleri tekrar veri çerçevesine birleştir:
combined['FT_tfidf'] = tfidf_sent_vectors

In [107]:
keyword = 'fasteners'

query = [combined.loc[combined.MRLineItemDescription.str.contains(keyword)].iloc[0]['FT_tfidf']]
query = np.array(list(query))
query = np.nan_to_num(query)

vectors = np.array(list(combined.FT_tfidf.values))
vectors = np.nan_to_num(vectors)

cosine_similarities = pd.Series(cosine_similarity(query, vectors).flatten())

for i,j in cosine_similarities.nlargest(10).iteritems():
  print(str(i) + '-' + combined.MRLineItemDescription.iloc[i] + " " + str(j))

3108-Metal Roof Panels (Gutter Downspouts with all fasteners) spec 74113 1.0
3109-Metal Wall Panel (CUSTOM STAINLESS STEEL CLADDING SYSTEM with all accessories and fasteners) spec 74213 0.9053069646681828
68056-Roof Screen Wall Panels Complete to include clips fasteners and any intermediate support required to maintain panel integrity. 0.8998648038409534
24940-Linear Wood Ceilings and Panels (WP3) - Aluratone 750 Microperf Walls Ceilings Soffits & Fascia's.  3/4 Thick x up to 4' Wide x upto 8' or 10' Long.  Species: Harward Veneer as per specifications QS/BM.  Finish Custom Stain to March Architect's Sample.  ASTME-8 Fire Rated.  Includes Z Clips Torsion Springs and Saddle Climps and heavy uty Metric Grid.   0.8959730978227042
24941-Panel Grille Ceilings (WPC3) - Panel Grille Ceilings as per Drawings - Wood Backed.  Species: Rulon Harwood as per specifications.  Finish: Custom Stain to Match Architect's Sample.  ASTME-84 Fire Rated.  Includes Black Fabric Backer Wood Backer Clips Heavy

In [113]:
print(combined.loc[combined.MRLineItemDescription.str.contains(keyword)]['MRLineItemDescription'].values[0])

Metal Wall Panel (CUSTOM STAINLESS STEEL CLADDING SYSTEM with all accessories and fasteners) spec 74213


In [0]:
keyword = 'A000'

query = [combined.loc[combined.CategoryCode.str.contains(keyword)].iloc[0]['FT_tfidf']]
query = np.array(list(query))
query = np.nan_to_num(query)

vectors = np.array(list(combined.FT_tfidf.values))
vectors = np.nan_to_num(vectors)

cosine_similarities = pd.Series(cosine_similarity(query, vectors).flatten())

for i,j in cosine_similarities.nlargest(10).iteritems():
  print(str(i) + '-' + combined.CategoryCode.iloc[i] + " " + str(j))

In [0]:
print(combined.loc[combined.MRLineItemDescription.str.contains(keyword)]['MRLineItemDescription'].values[0])

In [117]:
keyword = 'ATWP'

query = [combined.loc[combined.CommonCostCode.str.contains(keyword)].iloc[0]['FT_tfidf']]
query = np.array(list(query))
query = np.nan_to_num(query)

vectors = np.array(list(combined.FT_tfidf.values))
vectors = np.nan_to_num(vectors)

cosine_similarities = pd.Series(cosine_similarity(query, vectors).flatten())

for i,j in cosine_similarities.nlargest(10).iteritems():
  print(str(i) + '-' + combined.CommonCostCode.iloc[i] + " " + str(j))

26322-33ATWP 0.9999999999999998
26323-33ATWP 0.9999999999999998
398626-33AFFN 0.923457699201812
29333-33ATJS 0.9194347571755452
29434-33ATJS 0.9194347571755452
24589-33AMSC 0.9160184619556551
24747-33AMSC 0.9160184619556551
398618-33AFFN 0.9160102778487015
398627-33AFFN 0.9160102778487015
398635-33AFFN 0.9160102778487015


In [118]:
print(combined.loc[combined.CommonCostCode.str.contains(keyword)]['CommonCostCode'].values[0])

33ATWP


In [0]:
# combined = combined['FT_tfidf']
# combined.to_csv( "/content/drive/My Drive/tf-ıdf-fasttext/FT_tfidf.csv", index=False) 